In [170]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")
table = soup.find('table', class_='sortable')
ths = table.find_all('th')
headings = [th.text.strip() for th in ths]

neighborhoods = pd.DataFrame(columns=headings)

for tr in table.find_all('tr'):
    tds = tr.find_all('td')
    if not tds:
        continue
    x = [td.text.strip() for td in tds[:3]]
    if x[1] != "Not assigned":
        if x[2] == "Not assigned":
            x[2] = x[1]
        neighborhoods = neighborhoods.append(pd.Series(x, index=headings), ignore_index=True)
neighborhoods.rename(columns={"Community": "Borough", "Neighbourhood": "Neighborhood"}, inplace = True)

neighborhoods.head(5)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [171]:
coords = pd.read_csv("http://cocl.us/Geospatial_data")
coords.set_index("Postal Code", inplace=True)
neighborhoods=neighborhoods.join(coords, on="Postal Code", how="left")
neighborhoods.head()


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [172]:
neighborhoods = neighborhoods[neighborhoods['Borough'].str.contains("Toronto")]
neighborhoods.reset_index(drop=True, inplace=True)
neighborhoods.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [173]:
import numpy as np # library to handle data in a vectorized manner

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

 # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [174]:
CLIENT_ID = 'PHCGQZKCEA1TF1RF1XTFMVVJF4HJKV52GFK4HTSG5FVKBEVU' # your Foursquare ID
CLIENT_SECRET = 'TAVMWAFUYBGBXUWDKHVICRY5PTEVZBPVI2QXASM1K5VAKJKI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [175]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print([name, lat, lng])
        LIMIT=50
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [176]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

['Regent Park, Harbourfront', 43.6542599, -79.3606359]
["Queen's Park, Ontario Provincial Government", 43.6623015, -79.3894938]
['Garden District, Ryerson', 43.6571618, -79.37893709999999]
['St. James Town', 43.6514939, -79.3754179]
['The Beaches', 43.67635739999999, -79.2930312]
['Berczy Park', 43.644770799999996, -79.3733064]
['Central Bay Street', 43.6579524, -79.3873826]
['Christie', 43.669542, -79.4225637]
['Richmond, Adelaide, King', 43.65057120000001, -79.3845675]
['Dufferin, Dovercourt Village', 43.66900510000001, -79.4422593]
['Harbourfront East, Union Station, Toronto Islands', 43.6408157, -79.38175229999999]
['Little Portugal, Trinity', 43.647926700000006, -79.4197497]
['The Danforth West, Riverdale', 43.6795571, -79.352188]
['Toronto Dominion Centre, Design Exchange', 43.6471768, -79.38157640000001]
['Brockton, Parkdale Village, Exhibition Place', 43.6368472, -79.42819140000002]
['India Bazaar, The Beaches West', 43.6689985, -79.31557159999998]
['Commerce Court, Victoria Ho

In [177]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
neigh_col = toronto_onehot.pop('Neighborhood')
toronto_onehot.insert(0, 'Neighborhood', neigh_col)

toronto_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [178]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,0.00,0.020000,0.00,0.000000,0.000000,0.0000,0.0000,0.000000,0.0000,0.020000,0.040000,0.000000,0.000000,0.00,0.02,0.02,0.040000,0.000000,0.00,0.02,0.0000,0.000000,0.0000,0.00,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.00,0.00,0.0000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.040000,0.100000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.020000,0.0000,0.02,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.040000,0.000000,0.00,0.000000,0.020000,0.00,0.000000,0.00,0.00,0.02,0.020000,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.020000,0.02000

In [179]:
toronto_grouped_count = toronto_onehot[['Neighborhood','Italian Restaurant']].groupby('Neighborhood').mean().reset_index()
toronto_grouped_count

,Neighborhood,Italian Restaurant
0,Berczy Park,0.020000
1,"Brockton, Parkdale Village, Exhibition Place",0.043478
2,"Business reply mail Processing Centre, South C...",0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000
4,Central Bay Street,0.040000
5,Christie,0.062500
6,Church and Wellesley,0.000000
7,"Commerce Court, Victoria Hotel",0.000000
8,Davisville,0.060606
9,Davisville North,0.000000


In [180]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [181]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)


neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Beer Bar,Seafood Restaurant,Bakery,Farmers Market,Restaurant,Department Store,Breakfast Spot
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Nightclub,Coffee Shop,Climbing Gym,Burrito Place,Italian Restaurant,Intersection,Restaurant,Bar
2,"Business reply mail Processing Centre, South C...",Farmers Market,Gym / Fitness Center,Auto Workshop,Butcher,Restaurant,Burrito Place,Recording Studio,Fast Food Restaurant,Pizza Place,Comic Shop
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport,Bar,Harbor / Marina,Plane,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry
4,Central Bay Street,Coffee Shop,Bubble Tea Shop,Café,Italian Restaurant,Yoga Studio,Comic Shop,Sandwich Place,Burger Joint,Ramen Restaurant,Portuguese Restaurant


In [182]:
# set number of clusters
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([6, 6, 6, 6, 0, 6, 6, 6, 6, 6, 6, 6, 5, 6, 6, 6, 6, 6, 4, 6])

In [183]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted = neighborhoods_venues_sorted.join(toronto_grouped_count.set_index('Neighborhood'), on='Neighborhood')

toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head(20) # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Italian Restaurant
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Shoe Store,Brewery,0.000000
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Yoga Studio,Sushi Restaurant,Sandwich Place,Restaurant,Italian Restaurant,Diner,Café,Smoothie Shop,Beer Bar,0.029412
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,6,Coffee Shop,Clothing Store,Café,Fast Food Restaurant,Middle Eastern Restaurant,Bookstore,Theater,Ramen Restaurant,Sandwich Place,Spa,0.020000
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,6,Café,Coffee Shop,Gastropub,Cocktail Bar,Japanese Restaurant,Restaurant,Farmers Market,Beer Bar,Cosmetics Shop,Furniture / Home Store,0.020000
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,6,Trail,Pub,Health Food Store,Yoga Studio,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,0.000000
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,6,Coffee Shop,Cocktail Bar,Cheese Shop,Beer Bar,Seafood Restaurant,Bakery,Farmers Market,Restaurant,Department Store,Breakfast Spot,0.020000
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Bubble Tea Shop,Café,Italian Restaurant,Yoga Studio,Comic Shop,Sandwich Place,Burger Joint,Ramen Restaurant,Portuguese Restaurant,0.040000
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,6,Grocery Store,Café,Park,Restaurant,Candy Store,Athletics & Sports,Italian Restaurant,Baby Store,Coffee Shop,Nightclub,0.062500
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,6,Café,Coffee Shop,Steakhouse,Concert Hall,Restaurant,American Restaurant,Speakeasy,Hotel,Bar,Gym / Fitness Center,0.000000
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,6,Bakery,Pharmacy,Gym / Fitness Center,Supermarket,Bar,Middle Eastern Restaurant,Café,Bank,Grocery Store,Music Venue,0.000000


In [184]:
toronto_grouped_rest_freq = toronto_merged[['Cluster Labels', 'Italian Restaurant']].groupby('Cluster Labels').mean().reset_index()
toronto_grouped_rest_freq

,Cluster Labels,Italian Restaurant
0,0,0.023197
1,1,0.000000
2,2,0.000000
3,3,0.000000
4,4,0.000000
5,5,0.000000
6,6,0.022496


In [185]:
toronto_merged_0 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
toronto_merged_0

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Italian Restaurant
0,"Regent Park, Harbourfront",0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Shoe Store,Brewery,0.000000
1,"Queen's Park, Ontario Provincial Government",0,Coffee Shop,Yoga Studio,Sushi Restaurant,Sandwich Place,Restaurant,Italian Restaurant,Diner,Café,Smoothie Shop,Beer Bar,0.029412
6,Central Bay Street,0,Coffee Shop,Bubble Tea Shop,Café,Italian Restaurant,Yoga Studio,Comic Shop,Sandwich Place,Burger Joint,Ramen Restaurant,Portuguese Restaurant,0.040000
12,"The Danforth West, Riverdale",0,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Grocery Store,Pizza Place,Brewery,0.069767
23,"North Toronto West, Lawrence Park",0,Coffee Shop,Clothing Store,Bagel Shop,Furniture / Home Store,Ice Cream Shop,Fast Food Restaurant,Diner,Mexican Restaurant,Park,Chinese Restaurant,0.000000
24,"The Annex, North Midtown, Yorkville",0,Café,Sandwich Place,Coffee Shop,History Museum,Park,Pharmacy,Pizza Place,Pub,Middle Eastern Restaurant,Burger Joint,0.000000


In [186]:
# set number of clusters
kclusters = 2

toronto_grouped_clustering.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_grouped_0 = toronto_grouped_clustering.loc[toronto_grouped_clustering['Cluster Labels'] == 0]
toronto_grouped_0 = toronto_grouped_0.drop('Cluster Labels', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_0)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 0, 1])

In [187]:
toronto_merged_0 = toronto_merged_0.drop('Cluster Labels', 1)
toronto_merged_0.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged_0.sort_values(by='Cluster Labels', axis=0, inplace = True)
toronto_merged_0

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Italian Restaurant
23,0,"North Toronto West, Lawrence Park",Coffee Shop,Clothing Store,Bagel Shop,Furniture / Home Store,Ice Cream Shop,Fast Food Restaurant,Diner,Mexican Restaurant,Park,Chinese Restaurant,0.000000
0,1,"Regent Park, Harbourfront",Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Shoe Store,Brewery,0.000000
1,1,"Queen's Park, Ontario Provincial Government",Coffee Shop,Yoga Studio,Sushi Restaurant,Sandwich Place,Restaurant,Italian Restaurant,Diner,Café,Smoothie Shop,Beer Bar,0.029412
6,1,Central Bay Street,Coffee Shop,Bubble Tea Shop,Café,Italian Restaurant,Yoga Studio,Comic Shop,Sandwich Place,Burger Joint,Ramen Restaurant,Portuguese Restaurant,0.040000
12,1,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Grocery Store,Pizza Place,Brewery,0.069767
24,1,"The Annex, North Midtown, Yorkville",Café,Sandwich Place,Coffee Shop,History Museum,Park,Pharmacy,Pizza Place,Pub,Middle Eastern Restaurant,Burger Joint,0.000000


In [188]:
toronto_merged_0_group = toronto_merged_0[['Cluster Labels', 'Italian Restaurant']].groupby('Cluster Labels').mean().reset_index()
toronto_merged_0_group

,Cluster Labels,Italian Restaurant
0,0,0.000000
1,1,0.027836


In [189]:
toronto_merged_0_selected = toronto_merged_0.loc[(toronto_merged_0['Cluster Labels'] == 1)]
toronto_merged_0_selected = toronto_merged_0_selected.loc[toronto_merged_0['Italian Restaurant'] == 0.0]
toronto_merged_0_selected
                           

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Italian Restaurant
0,1,"Regent Park, Harbourfront",Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Shoe Store,Brewery,0.0
24,1,"The Annex, North Midtown, Yorkville",Café,Sandwich Place,Coffee Shop,History Museum,Park,Pharmacy,Pizza Place,Pub,Middle Eastern Restaurant,Burger Joint,0.0


In [146]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Italian Restaurant
33,Rosedale,1,Park,Playground,Trail,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,0.0


In [147]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Italian Restaurant
19,Roselawn,2,Garden,Music Venue,Yoga Studio,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,0.0


In [148]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Italian Restaurant
29,"Moore Park, Summerhill East",3,Playground,Trail,Yoga Studio,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,0.0


In [149]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Italian Restaurant
18,Lawrence Park,4,Swim School,Park,Bus Line,Yoga Studio,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,0.0


In [150]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Italian Restaurant
21,"Forest Hill North & West, Forest Hill Road Park",5,Jewelry Store,Trail,Sushi Restaurant,Park,Garden,Cuban Restaurant,Doner Restaurant,Dog Run,Distribution Center,Discount Store,0.0


In [151]:
toronto_merged_6= toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
toronto_merged_6

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Italian Restaurant
2,"Garden District, Ryerson",6,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Bookstore,Ramen Restaurant,Theater,Fast Food Restaurant,Pizza Place,Diner,0.020000
3,St. James Town,6,Café,Gastropub,Coffee Shop,Cocktail Bar,Cosmetics Shop,Restaurant,Farmers Market,Japanese Restaurant,Beer Bar,Bookstore,0.020000
4,The Beaches,6,Trail,Pub,Health Food Store,Yoga Studio,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,0.000000
5,Berczy Park,6,Coffee Shop,Farmers Market,Cocktail Bar,Restaurant,Cheese Shop,Beer Bar,Bakery,Seafood Restaurant,Hotel,Park,0.020000
7,Christie,6,Grocery Store,Café,Park,Candy Store,Athletics & Sports,Restaurant,Italian Restaurant,Coffee Shop,Nightclub,Baby Store,0.062500
8,"Richmond, Adelaide, King",6,Coffee Shop,Café,Concert Hall,American Restaurant,Steakhouse,Restaurant,Noodle House,Department Store,Smoke Shop,Seafood Restaurant,0.000000
9,"Dufferin, Dovercourt Village",6,Pharmacy,Bakery,Bank,Café,Park,Brewery,Music Venue,Supermarket,Bar,Grocery Store,0.000000
10,"Harbourfront East, Union Station, Toronto Islands",6,Coffee Shop,Aquarium,Hotel,Brewery,Plaza,Café,Park,Monument / Landmark,Sandwich Place,Salad Place,0.020000
11,"Little Portugal, Trinity",6,Bar,Coffee Shop,Café,Vietnamese Restaurant,Asian Restaurant,Restaurant,Men's Store,Japanese Restaurant,Juice Bar,Korean Restaurant,0.022222
13,"Toronto Dominion Centre, Design Exchange",6,Coffee Shop,Café,Restaurant,Seafood Restaurant,Hotel,Gastropub,Beer Bar,Bakery,Japanese Restaurant,Speakeasy,0.000000


In [154]:
kclusters=5
toronto_grouped_6 = toronto_grouped_clustering.loc[toronto_grouped_clustering['Cluster Labels'] == 6]
toronto_grouped_6 = toronto_grouped_6.drop('Cluster Labels', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_6)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([4, 4, 2, 1, 4, 1, 4, 2, 3, 1, 4, 4, 4, 1, 2, 1, 1, 1, 4, 1])

In [155]:
toronto_merged_6 = toronto_merged_6.drop('Cluster Labels', 1)
toronto_merged_6.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged_6.sort_values(by='Cluster Labels', axis=0, inplace = True)
toronto_merged_6

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Italian Restaurant
15,0,"India Bazaar, The Beaches West",Park,Gym,Sandwich Place,Italian Restaurant,Burrito Place,Liquor Store,Restaurant,Ice Cream Shop,Pub,Steakhouse,0.052632
20,1,Davisville North,Park,Hotel,Dog Run,Breakfast Spot,Gym / Fitness Center,Sandwich Place,Department Store,Dance Studio,Food & Drink Shop,Doner Restaurant,0.000000
16,1,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,Gym,Seafood Restaurant,Deli / Bodega,Beer Bar,Japanese Restaurant,Gastropub,0.000000
37,1,Church and Wellesley,Coffee Shop,Gay Bar,Restaurant,Yoga Studio,Men's Store,Japanese Restaurant,Burger Joint,Hobby Shop,Hotel,Distribution Center,0.000000
31,1,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,Supermarket,Fried Chicken Joint,Sports Bar,Sushi Restaurant,Bank,Restaurant,Pizza Place,Bagel Shop,Liquor Store,0.000000
3,1,St. James Town,Café,Gastropub,Coffee Shop,Cocktail Bar,Cosmetics Shop,Restaurant,Farmers Market,Japanese Restaurant,Beer Bar,Bookstore,0.020000
14,1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Nightclub,Coffee Shop,Climbing Gym,Burrito Place,Italian Restaurant,Intersection,Restaurant,Bar,0.043478
11,1,"Little Portugal, Trinity",Bar,Coffee Shop,Café,Vietnamese Restaurant,Asian Restaurant,Restaurant,Men's Store,Japanese Restaurant,Juice Bar,Korean Restaurant,0.022222
38,1,"Business reply mail Processing Centre, South C...",Brewery,Pizza Place,Recording Studio,Restaurant,Light Rail Station,Auto Workshop,Butcher,Burrito Place,Farmers Market,Gym / Fitness Center,0.000000
2,1,"Garden District, Ryerson",Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Bookstore,Ramen Restaurant,Theater,Fast Food Restaurant,Pizza Place,Diner,0.020000


In [156]:
toronto_merged_6_group = toronto_merged_6[['Cluster Labels', 'Italian Restaurant']].groupby('Cluster Labels').mean().reset_index()
toronto_merged_6_group

,Cluster Labels,Italian Restaurant
0,0,0.052632
1,1,0.018755
2,2,0.033605
3,3,0.000000
4,4,0.022511


In [167]:
toronto_merged_6_selected = toronto_merged_6.loc[(toronto_merged_6['Cluster Labels'] == 1)  | (toronto_merged_6['Cluster Labels'] == 4)]
toronto_merged_6_selected = toronto_merged_6_selected.loc[toronto_merged_6['Italian Restaurant'] == 0.0]
toronto_merged_6_selected
                             

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Italian Restaurant
20,1,Davisville North,Park,Hotel,Dog Run,Breakfast Spot,Gym / Fitness Center,Sandwich Place,Department Store,Dance Studio,Food & Drink Shop,Doner Restaurant,0.0
16,1,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,Gym,Seafood Restaurant,Deli / Bodega,Beer Bar,Japanese Restaurant,Gastropub,0.0
37,1,Church and Wellesley,Coffee Shop,Gay Bar,Restaurant,Yoga Studio,Men's Store,Japanese Restaurant,Burger Joint,Hobby Shop,Hotel,Distribution Center,0.0
31,1,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,Supermarket,Fried Chicken Joint,Sports Bar,Sushi Restaurant,Bank,Restaurant,Pizza Place,Bagel Shop,Liquor Store,0.0
38,1,"Business reply mail Processing Centre, South C...",Brewery,Pizza Place,Recording Studio,Restaurant,Light Rail Station,Auto Workshop,Butcher,Burrito Place,Farmers Market,Gym / Fitness Center,0.0
9,4,"Dufferin, Dovercourt Village",Pharmacy,Bakery,Bank,Café,Park,Brewery,Music Venue,Supermarket,Bar,Grocery Store,0.0
32,4,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport,Coffee Shop,Rental Car Location,Sculpture Garden,Harbor / Marina,Plane,Boat or Ferry,Boutique,0.0
30,4,"Kensington Market, Chinatown, Grange Park",Mexican Restaurant,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Bakery,Burger Joint,Dumpling Restaurant,Dessert Shop,Vietnamese Restaurant,Bar,0.0
4,4,The Beaches,Trail,Pub,Health Food Store,Yoga Studio,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,0.0
8,4,"Richmond, Adelaide, King",Coffee Shop,Café,Concert Hall,American Restaurant,Steakhouse,Restaurant,Noodle House,Department Store,Smoke Shop,Seafood Restaurant,0.0


In [193]:
toronto_merged_all = pd.concat([toronto_merged_0_selected, toronto_merged_6_selected])
toronto_merged_all

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Italian Restaurant
0,1,"Regent Park, Harbourfront",Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Shoe Store,Brewery,0.0
24,1,"The Annex, North Midtown, Yorkville",Café,Sandwich Place,Coffee Shop,History Museum,Park,Pharmacy,Pizza Place,Pub,Middle Eastern Restaurant,Burger Joint,0.0
20,1,Davisville North,Park,Hotel,Dog Run,Breakfast Spot,Gym / Fitness Center,Sandwich Place,Department Store,Dance Studio,Food & Drink Shop,Doner Restaurant,0.0
16,1,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,Gym,Seafood Restaurant,Deli / Bodega,Beer Bar,Japanese Restaurant,Gastropub,0.0
37,1,Church and Wellesley,Coffee Shop,Gay Bar,Restaurant,Yoga Studio,Men's Store,Japanese Restaurant,Burger Joint,Hobby Shop,Hotel,Distribution Center,0.0
31,1,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,Supermarket,Fried Chicken Joint,Sports Bar,Sushi Restaurant,Bank,Restaurant,Pizza Place,Bagel Shop,Liquor Store,0.0
38,1,"Business reply mail Processing Centre, South C...",Brewery,Pizza Place,Recording Studio,Restaurant,Light Rail Station,Auto Workshop,Butcher,Burrito Place,Farmers Market,Gym / Fitness Center,0.0
9,4,"Dufferin, Dovercourt Village",Pharmacy,Bakery,Bank,Café,Park,Brewery,Music Venue,Supermarket,Bar,Grocery Store,0.0
32,4,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport,Coffee Shop,Rental Car Location,Sculpture Garden,Harbor / Marina,Plane,Boat or Ferry,Boutique,0.0
30,4,"Kensington Market, Chinatown, Grange Park",Mexican Restaurant,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Bakery,Burger Joint,Dumpling Restaurant,Dessert Shop,Vietnamese Restaurant,Bar,0.0


In [194]:
toronto_merged_coords = toronto_merged_all.join(neighborhoods.set_index('Neighborhood'), on='Neighborhood')
toronto_merged_coords

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Italian Restaurant,Postal Code,Borough,Latitude,Longitude
0,1,"Regent Park, Harbourfront",Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Shoe Store,Brewery,0.0,M5A,Downtown Toronto,43.654260,-79.360636
24,1,"The Annex, North Midtown, Yorkville",Café,Sandwich Place,Coffee Shop,History Museum,Park,Pharmacy,Pizza Place,Pub,Middle Eastern Restaurant,Burger Joint,0.0,M5R,Central Toronto,43.672710,-79.405678
20,1,Davisville North,Park,Hotel,Dog Run,Breakfast Spot,Gym / Fitness Center,Sandwich Place,Department Store,Dance Studio,Food & Drink Shop,Doner Restaurant,0.0,M4P,Central Toronto,43.712751,-79.390197
16,1,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,Gym,Seafood Restaurant,Deli / Bodega,Beer Bar,Japanese Restaurant,Gastropub,0.0,M5L,Downtown Toronto,43.648198,-79.379817
37,1,Church and Wellesley,Coffee Shop,Gay Bar,Restaurant,Yoga Studio,Men's Store,Japanese Restaurant,Burger Joint,Hobby Shop,Hotel,Distribution Center,0.0,M4Y,Downtown Toronto,43.665860,-79.383160
31,1,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,Supermarket,Fried Chicken Joint,Sports Bar,Sushi Restaurant,Bank,Restaurant,Pizza Place,Bagel Shop,Liquor Store,0.0,M4V,Central Toronto,43.686412,-79.400049
38,1,"Business reply mail Processing Centre, South C...",Brewery,Pizza Place,Recording Studio,Restaurant,Light Rail Station,Auto Workshop,Butcher,Burrito Place,Farmers Market,Gym / Fitness Center,0.0,M7Y,East Toronto,43.662744,-79.321558
9,4,"Dufferin, Dovercourt Village",Pharmacy,Bakery,Bank,Café,Park,Brewery,Music Venue,Supermarket,Bar,Grocery Store,0.0,M6H,West Toronto,43.669005,-79.442259
32,4,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport,Coffee Shop,Rental Car Location,Sculpture Garden,Harbor / Marina,Plane,Boat or Ferry,Boutique,0.0,M5V,Downtown Toronto,43.628947,-79.394420
30,4,"Kensington Market, Chinatown, Grange Park",Mexican Restaurant,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Bakery,Burger Joint,Dumpling Restaurant,Dessert Shop,Vietnamese Restaurant,Bar,0.0,M5T,Downtown Toronto,43.653206,-79.400049


In [196]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="canada_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Toronto are 43.6534817, -79.3839347.


In [197]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_merged_coords['Latitude'], toronto_merged_coords['Longitude'], toronto_merged_coords['Borough'], toronto_merged_coords['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto